In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

In [ ]:
data

possible: Pclass, type of cabin, sex, age, age and gender

In [ ]:
data.corrwith(data['Survived'])

In [ ]:
a = {}
for c in data.columns:
    a[c]= data[c].isnull().sum()
    
a

In [ ]:
data.info()

In [ ]:
data.describe()

process data

* approach: 
* drop name, id, ticket (might cause noise. also i fail to see how that can be influential) 
* train vs val = 0.7/0.3
* random state = 0 
* for cabin, the letter is put into one hot encoder and the no stored as int, nan is median and most common value, respectively
* age is filled with mean 
* the rest is processed as either numerical data or one hot encoder

In [ ]:
testreal = test.copy()

In [ ]:
X = data.copy()
y = data['Survived']
X.drop(['Survived', 'Name', 'PassengerId', 'Ticket'], axis = 1, inplace = True)
test.drop(['Name', 'PassengerId', 'Ticket'], axis = 1, inplace = True)

In [ ]:
X['cabin_type']= X['Cabin'].str[0]
test['cabin_type']= test['Cabin'].str[0]

In [ ]:
def f(x):
    try:
        return np.float(x)
    except:
        return np.nan
    
X['cabin_no']= X['Cabin'].str[1:4].apply(f)
test['cabin_no']= test['Cabin'].str[1:4].apply(f)



In [ ]:
X.drop('Cabin', axis =1, inplace = True)
test.drop('Cabin', axis =1, inplace = True)

In [ ]:
X_t, X_v, y_t, y_v = train_test_split(X, y, train_size = 0.7, test_size= 0.3, random_state =0)

In [ ]:

num_cols= [cols for cols in X.columns if X[cols].dtypes in['int64', 'float64']]
    
num_cols

In [ ]:
cat_cols= [cols for cols in X.columns if not X[cols].dtypes in['int64', 'float64']]
cat_cols

In [ ]:
X['cabin_no'].value_counts()

In [ ]:
num_imputer = SimpleImputer(strategy = 'median')
cat_imputer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))]) 

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_imputer, num_cols),
        ('cat', cat_imputer, cat_cols)
    ])

In [ ]:
model = RandomForestClassifier(n_estimators=200, max_depth=6, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_t, y_t)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_v)

In [ ]:
score = mean_absolute_error(y_v, preds)
print('MAE:', score)

In [ ]:
test

In [ ]:
p = my_pipeline.predict(test)
sub = pd.DataFrame(p, index= testreal['PassengerId'], columns = ['Survived'])

In [ ]:
sub.to_csv('sub.csv')

In [ ]:
sub

In [ ]:
test